In [ ]:
%load_ext autoreload
%autoreload 2

# Creacion de Data aumentation

In [1]:
# Importar las librerías necesarias
import latam_airlines.utils.paths as path
from latam_airlines.utils.latam_utils import check_quality

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [ ]:
# leemos el dataset de training
data_train = pd.read_csv(path.data_processed_dir('data_train.csv'))
data_train.sample(5)

Para saber cuales serian las variables que mas pueden aporatar al modelo podemos hacer un test estadistico, Como la variable objetivo (y) es categórica, en lugar de utilizar un ANOVA para hacer una comparación de medias de las variables independientes, se puede utilizar un test estadístico para comparar la distribución de las variables independientes para cada categoría de la variable objetivo. Una opción popular es el test chi-cuadrado, que se puede calcular utilizando la función scipy.stats.chi2_contingency. Este test permite determinar si existe una asociación significativa entre la variable objetivo y cada una de las variables independientes.

In [ ]:
cols = data_train.drop('atraso_15', axis=1).columns
for col in cols:
    if col in ['DIA','MES','AÑO', 'HORA','MIN']:
        data_train[col] = data_train[col].astype(int).astype(str)
    else:
        data_train[col] = data_train[col].astype(str)

In [ ]:
from scipy.stats import chi2_contingency

p_value = []
cols = data_train.drop('atraso_15', axis=1).columns
for col in cols:
    # Crea las tablas de contingencia para cada variable independiente
    contingency_table_1 = pd.crosstab(data_train['atraso_15'], data_train[col])

    # Realiza la prueba de chi-cuadrado para cada tabla de contingencia
    chi2, p_value_1, dof, expected_values_1 = chi2_contingency(contingency_table_1)
    if p_value_1< 0.05:
        p_value.append({col:p_value_1})
        # Imprime los resultados
        print(f'P-value variable {col}: {p_value_1}')

Como resultados observamos que las variables escogidas son eimportante para poder predecir el atraso y nos aportan informacion relevante, a excepcion del anio, que no supero el test

In [ ]:
data_train.drop(columns='AÑO', inplace=True)
data_train.head(5)

In [ ]:
data_train.info()

In [ ]:
sns.catplot(
    data= data_train,
    x='atraso_15',
    kind='count',
)
plt.xlabel('Atraso mayor a 15 min')
plt.ylabel('Numero de Vuelos')
plt.title('conteo de vuelos atrasados en el 2017')
plt.show()

In [ ]:
col_order = ['DIA', 'MES', 'HORA', 'MIN','Vlo_change', 'Emp_change','temporada_alta','periodo_dia','DIANOM', 'MESNOM','Des-I', 'TIPOVUELO', 'OPERA','atraso_15']
data_train = data_train[col_order]

In [ ]:
# Obtiene las características y la variable objetivo
X = data_train.drop("atraso_15", axis=1)
y = data_train["atraso_15"]

# Crear una pipeline que incluya el ColumnTransformer
# Crear un objeto ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler( with_mean=False), ['DIA', 'MES', 'HORA', 'MIN','Vlo_change', 'Emp_change','temporada_alta']),
        ('cat-nominal', OneHotEncoder(), ['periodo_dia','DIANOM', 'MESNOM','Des-I', 'TIPOVUELO', 'OPERA'])
    ],
    )

#transformar datos
X_transform = preprocessor.fit_transform(X)

# Aplica SMOTE para generar datos sintéticos
smote = SMOTE(sampling_strategy='minority')
X_resampled, y_resampled = smote.fit_resample(X_transform, y)

# Inversión de la transformación de escalamiento
scaler = preprocessor.named_transformers_['num']
X_scaled = scaler.inverse_transform(X_resampled[:, :7])

# Inversión de la transformación One-hot
onehot = preprocessor.named_transformers_['cat-nominal']
X_onehot = X_resampled[:, 7:]
X_original = onehot.inverse_transform(X_onehot)

# Concatena los valores originales y convierte el resultado en un DataFrame
X_recovered = np.concatenate([X_scaled.toarray().astype(int), X_original], axis=1)

In [ ]:
data_train_balanced = pd.DataFrame(X_recovered, columns=X.columns)
data_train_balanced["atraso_15"] = y_resampled

In [ ]:
sns.catplot(
    data= data_train_balanced,
    x='atraso_15',
    kind='count',
)
plt.xlabel('Atraso mayor a 15 min')
plt.ylabel('Numero de Vuelos')
plt.title('conteo de vuelos atrasados en el 2017')
plt.show()

In [ ]:
check_quality(data_train_balanced)

In [ ]:
data_train_balanced.drop_duplicates(inplace=True)

In [ ]:
# Guarda el dataset balanceado en un archivo
data_train_balanced.to_csv(path.data_processed_dir('data_train_balanced.csv'), index=False)